# Reads the pretrained model.

In [3]:

from __future__ import print_function

import os 
os.chdir("C:/Users/Henry_B/work/sphere")
os.getcwd()

import torch
from torch.autograd import Variable
torch.backends.cudnn.bencmark = True

import os,sys,cv2,random,datetime
import numpy as np

from matlab_cp2tform import get_similarity_transform_for_cv2
import net_sphere
import zipfile

import re
import pandas as pd

def alignment(src_img,src_pts):
    ref_pts = [ [30.2946, 51.6963],[65.5318, 51.5014],
        [48.0252, 71.7366],[33.5493, 92.3655],[62.7299, 92.2041] ]
    crop_size = (96, 112)
    src_pts = np.array(src_pts).reshape(5,2)

    s = np.array(src_pts).astype(np.float32)
    r = np.array(ref_pts).astype(np.float32)

    tfm = get_similarity_transform_for_cv2(s, r)
    face_img = cv2.warpAffine(src_img, tfm, crop_size)
    return face_img

net = net_sphere.sphere20a()
net.load_state_dict(torch.load('model/sphere20a_20171020.pth'))
net.cuda()
net.eval()
net.feature = True # bypasses the laast FC to output features



# The result of bicubic interpolotion.

In [7]:

zfile = zipfile.ZipFile("data/lfw.zip")

landmark = {}
with open('data/lfw_landmark.txt') as f:
    landmark_lines = f.readlines()
for line in landmark_lines:
    l = line.replace('\n','').split('\t')
    landmark[l[0]] = [int(k) for k in l[1:]]

with open('data/pairs.txt') as f:
    pairs_lines = f.readlines()[1:]

cosdistance = []
samples = 6000
for _,_,files in os.walk("data/lfw"):
    for imgname in files:
        person = re.match("[^0-9]*", imgname)
        person = person.group(0)
        person = person[0:-1]
        imgname =person+"/"+imgname
        img = alignment(cv2.imdecode(np.frombuffer(zfile.read(imgname),np.uint8),1),landmark[imgname])
       
        
        img1 = cv2.resize(img, (24,28),fx=0,fy=0, interpolation =cv2.INTER_CUBIC )
        img1 = cv2.resize(img1, (96,112),fx=0,fy=0, interpolation =cv2.INTER_CUBIC )
        
        """
        cv2.imshow(imgname, img)
        cv2.imshow("bicubic", img1)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        """
        imglist = [img, cv2.flip(img, 1), img1, cv2.flip(img1, 1)]
        for i in range(len(imglist)):
            imglist[i] = imglist[i].transpose(2, 0, 1).reshape((1,3,112,96))
            imglist[i] = (imglist[i]-127.5)/128.0
        
        img = np.vstack(imglist)
        with torch.no_grad():
            img = Variable(torch.from_numpy(img).float()).cuda()
        output = net(img)
        f = output.data
        f0, f1 = f[0]+f[1],f[2]+f[3]
        tmp = f0.dot(f1)/(f0.norm()*f1.norm()+1e-5)
        cosdistance.append(tmp.item())
        if(len(cosdistance)%500 == 0):
            print(len(cosdistance))
            if(len(cosdistance)>samples):
                break
df = pd.DataFrame(np.array(cosdistance))
print(df.describe())


#img is np.array of shape (batch,3,112,96)
#lfw is of 250 250, lfw_eval alignment() resizes
# using     matlab_cp2tform.get_similarity_transform_for_cv2(s, r)
#    and cv2.warpAffine(src_img, tfm, crop_size)
# the self.fc5 (the first FC) requires the resolution!

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
                 0
count  6500.000000
mean      0.771156
std       0.077258
min       0.218228
25%       0.727591
50%       0.780157
75%       0.825200
max       0.953715


# The result of flipping 

In [8]:

zfile = zipfile.ZipFile("data/lfw.zip")

landmark = {}
with open('data/lfw_landmark.txt') as f:
    landmark_lines = f.readlines()
for line in landmark_lines:
    l = line.replace('\n','').split('\t')
    landmark[l[0]] = [int(k) for k in l[1:]]

with open('data/pairs.txt') as f:
    pairs_lines = f.readlines()[1:]

cosdistance = []
samples = 6000
for _,_,files in os.walk("data/lfw"):
    for imgname in files:
        person = re.match("[^0-9]*", imgname)
        person = person.group(0)
        person = person[0:-1]
        imgname =person+"/"+imgname
        img = alignment(cv2.imdecode(np.frombuffer(zfile.read(imgname),np.uint8),1),landmark[imgname])
       
        img1 = cv2.flip(img, 0)
        imglist = [img, img1]
        for i in range(len(imglist)):
            imglist[i] = imglist[i].transpose(2, 0, 1).reshape((1,3,112,96))
            imglist[i] = (imglist[i]-127.5)/128.0
        
        img = np.vstack(imglist)
        with torch.no_grad():
            img = Variable(torch.from_numpy(img).float()).cuda()
        output = net(img)
        f = output.data
        f0, f1 = f[0],f[1]
        tmp = f0.dot(f1)/(f0.norm()*f1.norm()+1e-5)
        cosdistance.append(tmp.item())
        if(len(cosdistance)%500 == 0):
            print(len(cosdistance))
            if(len(cosdistance)>samples):
                break
df = pd.DataFrame(np.array(cosdistance))
print(df.describe())


#img is np.array of shape (batch,3,112,96)
#lfw is of 250 250, lfw_eval alignment() resizes
# using     matlab_cp2tform.get_similarity_transform_for_cv2(s, r)
#    and cv2.warpAffine(src_img, tfm, crop_size)
# the self.fc5 (the first FC) requires the resolution!

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
                 0
count  6500.000000
mean      0.026688
std       0.107153
min      -0.436076
25%      -0.045648
50%       0.026181
75%       0.097964
max       0.447635


# The result of different instances of one person

In [6]:

zfile = zipfile.ZipFile("data/lfw.zip")

landmark = {}
with open('data/lfw_landmark.txt') as f:
    landmark_lines = f.readlines()
for line in landmark_lines:
    l = line.replace('\n','').split('\t')
    landmark[l[0]] = [int(k) for k in l[1:]]

with open('data/pairs.txt') as f:
    pairs_lines = f.readlines()[1:]

cosdistance = []
samples = 6000
for _,_,files in os.walk("data/lfw"):
    for imgname in files:
        person = re.match("[^0-9]*", imgname)
        person = person.group(0)
        person = person[0:-1]
        imgname =person+"/"+imgname
        
        imgnum = re.search(r"\d+", imgname).group(0)
        if imgnum == "0002":
            imgname1 = re.sub("0002", "0001" ,imgname)
          
            img = alignment(cv2.imdecode(np.frombuffer(zfile.read(imgname),np.uint8),1),landmark[imgname])
            img1 = alignment(cv2.imdecode(np.frombuffer(zfile.read(imgname1),np.uint8),1),landmark[imgname1])

            """
            cv2.imshow(imgname, img)         
            cv2.imshow(imgname1, img1)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            """
            
            imglist = [img, cv2.flip(img, 1), img1, cv2.flip(img1, 1)]
            for i in range(len(imglist)):
                imglist[i] = imglist[i].transpose(2, 0, 1).reshape((1,3,112,96))
                imglist[i] = (imglist[i]-127.5)/128.0

            img = np.vstack(imglist)
            with torch.no_grad():
                img = Variable(torch.from_numpy(img).float()).cuda()
            output = net(img)
            f = output.data
            f0, f1 = f[0]+f[1],f[2]+f[3]
            tmp = f0.dot(f1)/(f0.norm()*f1.norm()+1e-5)
            cosdistance.append(tmp.item())
            if(len(cosdistance)%500 == 0):
                print(len(cosdistance))
                if(len(cosdistance)>samples):
                    break
df = pd.DataFrame(np.array(cosdistance))
print(df.describe())



500
1000
1500
                 0
count  1680.000000
mean      0.658332
std       0.127805
min      -0.033425
25%       0.583516
50%       0.669929
75%       0.746939
max       0.967443


# Random image pairs

In [10]:

zfile = zipfile.ZipFile("data/lfw.zip")

landmark = {}
with open('data/lfw_landmark.txt') as f:
    landmark_lines = f.readlines()
for line in landmark_lines:
    l = line.replace('\n','').split('\t')
    landmark[l[0]] = [int(k) for k in l[1:]]

with open('data/pairs.txt') as f:
    pairs_lines = f.readlines()[1:]

cosdistance = []
samples = 6000
imgnames = []
for _,_,files in os.walk("data/lfw"):
    for imgname in files:
        person = re.match("[^0-9]*", imgname)
        person = person.group(0)
        person = person[0:-1]
        imgname =person+"/"+imgname
        imgnames.append(imgname)
        
for imgname in imgnames:    
        img = alignment(cv2.imdecode(np.frombuffer(zfile.read(imgname),np.uint8),1),landmark[imgname])
       
        tmp = np.random.randint(len(imgnames))
        img1 = alignment(cv2.imdecode(np.frombuffer(zfile.read(imgnames[tmp]),np.uint8),1),landmark[imgnames[tmp]])
        imglist = [img, img1]
        for i in range(len(imglist)):
            imglist[i] = imglist[i].transpose(2, 0, 1).reshape((1,3,112,96))
            imglist[i] = (imglist[i]-127.5)/128.0
        
        img = np.vstack(imglist)
        with torch.no_grad():
            img = Variable(torch.from_numpy(img).float()).cuda()
        output = net(img)
        f = output.data
        f0, f1 = f[0],f[1]
        tmp = f0.dot(f1)/(f0.norm()*f1.norm()+1e-5)
        cosdistance.append(tmp.item())
        if(len(cosdistance)%500 == 0):
            print(len(cosdistance))
            if(len(cosdistance)>samples):
                break
df = pd.DataFrame(np.array(cosdistance))
print(df.describe())


#img is np.array of shape (batch,3,112,96)
#lfw is of 250 250, lfw_eval alignment() resizes
# using     matlab_cp2tform.get_similarity_transform_for_cv2(s, r)
#    and cv2.warpAffine(src_img, tfm, crop_size)
# the self.fc5 (the first FC) requires the resolution!

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
                 0
count  6500.000000
mean      0.038778
std       0.124721
min      -0.367741
25%      -0.043736
50%       0.033764
75%       0.115174
max       0.834991
